In [2]:
from sympy import *

th1, th2, th3 = symbols('θ1 θ2 θ3')
L = symbols('L')

p = Matrix([
    L*cos(th1)*(cos(th2 + th3) + cos(th2)),
    L*sin(th1)*(cos(th2 + th3) + cos(th2)),
    L*(1 - sin(th2) - sin(th2 + th3))
])

dp_dq1 = diff(p, th1)
dp_dq2 = diff(p, th2)
dp_dq3 = diff(p, th3)

#jacobian:
Jv = simplify(dp_dq1.row_join(dp_dq2).row_join(dp_dq3))
Jv

Matrix([
[-L*(cos(θ2) + cos(θ2 + θ3))*sin(θ1), -L*(sin(θ2) + sin(θ2 + θ3))*cos(θ1), -L*sin(θ2 + θ3)*cos(θ1)],
[ L*(cos(θ2) + cos(θ2 + θ3))*cos(θ1), -L*(sin(θ2) + sin(θ2 + θ3))*sin(θ1), -L*sin(θ1)*sin(θ2 + θ3)],
[                                  0,         -L*(cos(θ2) + cos(θ2 + θ3)),         -L*cos(θ2 + θ3)]])

In [3]:
th1, th2, th3 = symbols('θ1 θ2 θ3')
L = symbols('L')

DHs = [
    #a, α, d, θ
    [0, -pi/2, L, th1],
    [L, 0, 0, th2],
    [L, 0, 0, th3]
]
HTMs = [eye(4)] #start with base frame, will append more
for i in range(len(DHs)):
    DH = DHs[i]
    a, alpha, d, theta = DH
    Rot_th = Matrix([
        [cos(theta), -sin(theta), 0, 0],
        [sin(theta), cos(theta), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    Trans_d = Matrix([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, d],
        [0, 0, 0, 1]
    ])
    Trans_a = Matrix([
        [1, 0, 0, a],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    Rot_alpha = Matrix([
        [1, 0, 0, 0],
        [0, cos(alpha), -sin(alpha), 0],
        [0, sin(alpha), cos(alpha), 0],
        [0, 0, 0, 1]
    ])
    HTM = Rot_th @ Trans_d @ Trans_a @ Rot_alpha
    HTMs.append(simplify(HTMs[-1] @ HTM))

#get a 3x3 rotation and 3x1 displacement from a 4x4 HTM
def extract_R_p(T):
    R = T[0:3, 0:3]
    p = T[0:3, 3]
    return R, p

#add a 3x1 displacement to a 4x4 HTM
def displace(T, p):
    displacement = Matrix([
        [1, 0, 0, p[0]],
        [0, 1, 0, p[1]],
        [0, 0, 1, p[2]],
        [0, 0, 0, 1]
    ])
    return displacement @ T

#Center of mass forward kinematics
c1, c2, c3 = symbols('c1 c2 c3')
rc1_1 = Matrix([0, c1, 0])
rc2_2 = Matrix([-c2, 0, 0])
rc3_3 = Matrix([-c3, 0, 0])

R1, _ = extract_R_p(HTMs[1])
R2, _ = extract_R_p(HTMs[2])
R3, _ = extract_R_p(HTMs[3])

HTM_c1 = simplify(displace(HTMs[1], R1@rc1_1))
HTM_c2 = simplify(displace(HTMs[2], R2@rc2_2))
HTM_c3 = simplify(displace(HTMs[3], R3@rc3_3))

In [4]:
HTM_c1

Matrix([
[cos(θ1),  0, -sin(θ1),      0],
[sin(θ1),  0,  cos(θ1),      0],
[      0, -1,        0, L - c1],
[      0,  0,        0,      1]])

In [5]:
HTM_c2

Matrix([
[cos(θ1)*cos(θ2), -sin(θ2)*cos(θ1), -sin(θ1),    (L - c2)*cos(θ1)*cos(θ2)],
[sin(θ1)*cos(θ2), -sin(θ1)*sin(θ2),  cos(θ1),    (L - c2)*sin(θ1)*cos(θ2)],
[       -sin(θ2),         -cos(θ2),        0, -L*sin(θ2) + L + c2*sin(θ2)],
[              0,                0,        0,                           1]])

In [6]:
HTM_c3

Matrix([
[cos(θ1)*cos(θ2 + θ3), -sin(θ2 + θ3)*cos(θ1), -sin(θ1), (L*(cos(θ2) + cos(θ2 + θ3)) - c3*cos(θ2 + θ3))*cos(θ1)],
[sin(θ1)*cos(θ2 + θ3), -sin(θ1)*sin(θ2 + θ3),  cos(θ1), (L*(cos(θ2) + cos(θ2 + θ3)) - c3*cos(θ2 + θ3))*sin(θ1)],
[       -sin(θ2 + θ3),         -cos(θ2 + θ3),        0,      -L*(sin(θ2) + sin(θ2 + θ3) - 1) + c3*sin(θ2 + θ3)],
[                   0,                     0,        0,                                                      1]])

In [77]:
#Get position of each COM then take derivative to get Jacobian
c1_d1 = extract_R_p(HTM_c1)[1].diff(th1)
c1_d2 = extract_R_p(HTM_c1)[1].diff(th2)
c1_d3 = extract_R_p(HTM_c1)[1].diff(th3)
Jvc1 = simplify(c1_d1.row_join(c1_d2).row_join(c1_d3))

c2_d1 = extract_R_p(HTM_c2)[1].diff(th1)
c2_d2 = extract_R_p(HTM_c2)[1].diff(th2)
c2_d3 = extract_R_p(HTM_c2)[1].diff(th3)
Jvc2 = simplify(c2_d1.row_join(c2_d2).row_join(c2_d3))

c3_d1 = extract_R_p(HTM_c3)[1].diff(th1)
c3_d2 = extract_R_p(HTM_c3)[1].diff(th2)
c3_d3 = extract_R_p(HTM_c3)[1].diff(th3)
Jvc3 = simplify(c3_d1.row_join(c3_d2).row_join(c3_d3))

q1_dot, q2_dot, q3_dot = symbols('\dot{q1} \dot{q2} \dot{q3}') 
qdots = Matrix([q1_dot, q2_dot, q3_dot])

vc1 = Jvc1 @ qdots
vc2 = Jvc2 @ qdots
vc3 = Jvc3 @ qdots

In [79]:
vc1

Matrix([
[0],
[0],
[0]])

In [80]:
vc2

Matrix([
[\dot{q1}*(-L + c2)*sin(q1)*cos(q2) + \dot{q2}*(-L + c2)*sin(q2)*cos(q1)],
[ \dot{q1}*(L - c2)*cos(q1)*cos(q2) + \dot{q2}*(-L + c2)*sin(q1)*sin(q2)],
[                                             \dot{q2}*(-L + c2)*cos(q2)]])

In [81]:
vc3

Matrix([
[-\dot{q1}*(L*(cos(q2) + cos(q2 + q3)) - c3*cos(q2 + q3))*sin(q1) - \dot{q2}*(L*(sin(q2) + sin(q2 + q3)) - c3*sin(q2 + q3))*cos(q1) + \dot{q3}*(-L + c3)*sin(q2 + q3)*cos(q1)],
[ \dot{q1}*(L*(cos(q2) + cos(q2 + q3)) - c3*cos(q2 + q3))*cos(q1) - \dot{q2}*(L*(sin(q2) + sin(q2 + q3)) - c3*sin(q2 + q3))*sin(q1) + \dot{q3}*(-L + c3)*sin(q1)*sin(q2 + q3)],
[                                                                                  \dot{q2}*(-L*(cos(q2) + cos(q2 + q3)) + c3*cos(q2 + q3)) + \dot{q3}*(-L + c3)*cos(q2 + q3)]])

In [84]:
zeros

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

In [101]:
u = Matrix([0, 0, 1]) #all joints rotate about their z axis
z3 = Matrix([0, 0, 0]) #3x1 zero vector

Jω1 = u.row_join(z3).row_join(z3)
Jω2 = u.row_join(R1 @ u).row_join(z3)
Jω3 = u.row_join(R1 @ u).row_join(R2 @ u)

ω1 = simplify(Jω1 @ qdots)
ω2 = simplify(Jω2 @ qdots)
ω3 = simplify(Jω3 @ qdots)

In [102]:
ω1

Matrix([
[       0],
[       0],
[\dot{q1}]])

In [103]:
ω2

Matrix([
[-\dot{q2}*sin(q1)],
[ \dot{q2}*cos(q1)],
[         \dot{q1}]])

In [104]:
ω3

Matrix([
[(-\dot{q2} - \dot{q3})*sin(q1)],
[ (\dot{q2} + \dot{q3})*cos(q1)],
[                      \dot{q1}]])